## Quantifying uncertainty in the MACE model: bootstrapping and deep ensembles. 


You are offered to validate the results obtained previously, outlined in "Strategies of uncertainty quantification for graph-neural-network-based interatomic potentials" by M. Radova. The report contains details about the dataset, MACE architecture, computational details and information on the chosen UQ strategies, which are deep ensembles and bootstrapped ensembles. It is recommended to read the report to understand the background of the offered task. 
For this task, you are not required to train any models, as it would require ~24h, if the 10 ensemble members are trained simultaneously. 

You are provided with 2 pre-trained ensembles, which you have to evaluate and write the predicted structures into "x_predictions.npz" files. 
For both deep and bootstrapped ensembles, the hyperparameters stated in the report were used, apart from the list of random seeds for deep ensembles, which was regenerated, and is as follows: split1: 321, split2: 5667, split3: 33390, split4: 43, split5: 999.

For the bootstrapped ensemble, test/train/validation splits were regenerated, to ensure the difference to the original bootstrapped ensemble. Importantly, the boostrapped ensemble is trained on the same random seed, which is a default 123. 

To obtain the standard deviations on the data, for each data point, predictions are extracted from each ensemble member, and the mean and variance are calculated. 

### 1. Install MACE on your SCRTP machine

**Load modules:**

module purge

module load GCC OpenMPI torchvision/0.13.1-CUDA-11.7.0

**Create a virtual environment for MACE:**

python3 -m venv ~/maceenv

source ~/maceenv/bin/activate

**Clone repository and install from folder:**

git clone --recursive https://github.com/ACEsuit/mace.git

pip install ./mace




### 2. Clone the repository into your SCRTP machine
git clone  https://github.com/7radians/gnn_uq.git

Run this notebook from an SCRTP CPU node. Typing "jupyter notebook" should be enough, if Jupyter is installed. 

### 3. Evaluate the models

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from colorama import Fore

from ase.io import write, read
from ase import Atoms
from mace.calculators import MACECalculator
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

In [4]:
#Write model predictions for the bootstrapped ensemble
options = ['b1', 'b2', 'b3', 'b4', 'b5' ] 
folder = 'b'
for i, option in enumerate(options):
    energy_pred = []
    forces_pred = []

    db = read(f'{folder}/{folder}_eval.xyz@:') # read db with test dataset
        
    calculator = MACECalculator(model_path=f'{folder}/MACE_model_{option}.model', device='cpu')

    for atoms in db:
    
        cur_atom = Atoms(atoms.symbols, positions=atoms.positions, cell=atoms.cell, pbc=atoms.pbc)
        cur_atom.set_calculator(calculator)
        energy_pred.append(cur_atom.get_potential_energy())
        forces_pred.append(cur_atom.get_forces()[18:])

    np.savez(f'{folder}/{option}_predictions.npz',energy=energy_pred,forces=forces_pred)


In [7]:
#Write model predictions for the deep ensemble
options = ['f1', 'f2', 'f3', 'f4', 'f5' ] 
folder = 'f'
for i, option in enumerate(options):

    energy_pred = []
    forces_pred = []

    db = read(f'{folder}/f_eval.xyz@:') # read db with test dataset
        
    calculator = MACECalculator(model_path=f'{folder}/MACE_model_{option}.model', device='cpu')

    for atoms in db:
        cur_atom = Atoms(atoms.symbols, positions=atoms.positions, cell=atoms.cell, pbc=atoms.pbc)
        cur_atom.set_calculator(calculator)
        energy_pred.append(cur_atom.get_potential_energy())
        forces_pred.append(cur_atom.get_forces()[18:])

    np.savez(f'{folder}/{option}_predictions.npz',energy=energy_pred,forces=forces_pred)

### 4. Perform the UQ

In [8]:
#Bootstraps UQ

option = 'b'
property_str = 'energy'

prediction1 = np.load(f'{option}/b1_predictions.npz',allow_pickle=True)[property_str].tolist()
prediction2 = np.load(f'{option}/b2_predictions.npz',allow_pickle=True)[property_str].tolist()
prediction3 = np.load(f'{option}/b3_predictions.npz',allow_pickle=True)[property_str].tolist()
prediction4 = np.load(f'{option}/b4_predictions.npz',allow_pickle=True)[property_str].tolist()
prediction5 = np.load(f'{option}/b5_predictions.npz',allow_pickle=True)[property_str].tolist()

num = 5
length = len(prediction1)
mean_energies = np.zeros(length)
e_squared_diffs = np.zeros((length, num))
e_variances = np.zeros(length)
e_std = np.zeros(length)


pred1 = np.zeros(length)
pred2 = np.zeros(length)
pred3 = np.zeros(length)
pred4 = np.zeros(length)
pred5 = np.zeros(length)

if property_str=='energy':
    pm1 = min(prediction1)
    pm2 = min(prediction2)
    pm3 = min(prediction3)
    pm4 = min(prediction4)
    pm5 = min(prediction5)
    for i in range(length):
        pred1[i] = prediction1[i] - pm1
        pred2[i] = prediction2[i] - pm2
        pred3[i] = prediction3[i] - pm3
        pred4[i] = prediction4[i] - pm4
        pred5[i] = prediction5[i] - pm5
        
    
if property_str=='energy':
    for i in range(length):
           preds = np.array([float(pred1[i]), float(pred2[i]), float(pred3[i]), float(pred4[i]), float(pred5[i])]) 
           mean_energies[i] = np.mean(preds, dtype=np.float64)
        
  
if property_str=='energy':
    for i in range(length):
           j=0
           for pred in [pred1, pred2, pred3, pred4, pred5]:
               e_squared_diffs[i,j] = (pred[i] - mean_energies[i])**2
               j=j+1
           e_variances[i] = np.mean(e_squared_diffs[i,:], axis=0)
           e_std[i] = np.sqrt(e_variances[i])

#Uncomment to see the distribution
#sns.histplot(e_std,stat="probability")
#plt.xlabel('σ / eV')
#plt.ylabel('Probability')
#plt.title('UQ of energy prediction - bootstrapping')
#plt.grid(True)
#plt.savefig('e_uq_b.png')

b_std = np.mean(e_std)

print("Mean σ, eV :", b_std)


Mean σ, eV : 0.005910500328955989


In [9]:
#Deep ensembles UQ


option = 'f'
property_str = 'energy'

prediction1 = np.load(f'{option}/f1_predictions.npz',allow_pickle=True)[property_str].tolist()
prediction2 = np.load(f'{option}/f2_predictions.npz',allow_pickle=True)[property_str].tolist()
prediction3 = np.load(f'{option}/f3_predictions.npz',allow_pickle=True)[property_str].tolist()
prediction4 = np.load(f'{option}/f4_predictions.npz',allow_pickle=True)[property_str].tolist()
prediction5 = np.load(f'{option}/f5_predictions.npz',allow_pickle=True)[property_str].tolist()

num = 5
length = len(prediction1)
mean_energies = np.zeros(length)
e_squared_diffs = np.zeros((length, num))
e_variances = np.zeros(length)
e_std = np.zeros(length)


pred1 = np.zeros(length)
pred2 = np.zeros(length)
pred3 = np.zeros(length)
pred4 = np.zeros(length)
pred5 = np.zeros(length)

if property_str=='energy':
    pm1 = min(prediction1)
    pm2 = min(prediction2)
    pm3 = min(prediction3)
    pm4 = min(prediction4)
    pm5 = min(prediction5)
    for i in range(length):
        pred1[i] = prediction1[i] - pm1
        pred2[i] = prediction2[i] - pm2
        pred3[i] = prediction3[i] - pm3
        pred4[i] = prediction4[i] - pm4
        pred5[i] = prediction5[i] - pm5
        
    
if property_str=='energy':
    for i in range(length):
           preds = np.array([float(pred1[i]), float(pred2[i]), float(pred3[i]), float(pred4[i]), float(pred5[i])]) 
           mean_energies[i] = np.mean(preds, dtype=np.float64)
        
  
if property_str=='energy':
    for i in range(length):
           j=0
           for pred in [pred1, pred2, pred3, pred4, pred5]:
               e_squared_diffs[i,j] = (pred[i] - mean_energies[i])**2
               j=j+1
           e_variances[i] = np.mean(e_squared_diffs[i,:], axis=0)
           e_std[i] = np.sqrt(e_variances[i])

#Uncomment to see the distribution
#sns.histplot(e_std,stat="probability")
#plt.xlabel('σ / eV')
#plt.ylabel('Probability')
#plt.title('UQ of energy prediction - deep ensembles')
#plt.grid(True)
#plt.savefig('e_uq_f.png')

f_std = np.mean(e_std)

print("Mean σ, eV :", f_std)

Mean σ, eV : 0.009562758513155504


### 5. Compare the results with the report 

In [10]:
print(Fore.BLUE + 'Reported results:')
print(Fore.BLACK + 'Mean σ, eV for the bootstrapped ensemble:',  0.005487420202870646)
print(Fore.BLACK  + 'Mean σ, eV for the deep ensemble:',  0.009678556148518234)

print(Fore.RED + 'Your results:')
print(Fore.BLACK + 'Mean σ, eV for the bootstrapped ensemble:',  b_std)
print(Fore.BLACK  + 'Mean σ, eV for the deep ensemble:',  f_std)


Reported results:
Mean σ, eV for the bootstrapped ensemble: 0.005487420202870646
Mean σ, eV for the deep ensemble: 0.009678556148518234
Your results:
Mean σ, eV for the bootstrapped ensemble: 0.005910500328955989
Mean σ, eV for the deep ensemble: 0.009562758513155504
